In [6]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [7]:
def getCurrentWeather(params):
    weather = 'http://api.openweathermap.org/data/2.5/weather'
    params['APPID'] = os.environ['APPID']
    params['units'] = 'metric'
    return requests.get(weather,params)

In [8]:
params = {'q':'Paris'}

In [9]:
response = getCurrentWeather(params)

In [10]:
response.status_code

200

In [11]:
response.json()

{'base': 'stations',
 'clouds': {'all': 48},
 'cod': 200,
 'coord': {'lat': 48.85, 'lon': 2.35},
 'dt': 1586738716,
 'id': 2988507,
 'main': {'feels_like': 9.91,
  'humidity': 62,
  'pressure': 1012,
  'temp': 13.31,
  'temp_max': 16,
  'temp_min': 10},
 'name': 'Paris',
 'rain': {'3h': 0.68},
 'sys': {'country': 'FR',
  'id': 6550,
  'sunrise': 1586754169,
  'sunset': 1586803128,
  'type': 1},
 'timezone': 7200,
 'visibility': 10000,
 'weather': [{'description': 'light rain',
   'icon': '10n',
   'id': 500,
   'main': 'Rain'}],
 'wind': {'deg': 40, 'speed': 3.6}}

## Getting World Cities

In [9]:
world_cities = requests.get('https://worldpopulationreview.com/world-cities/')

In [10]:
cities = BeautifulSoup(world_cities.text,'lxml')

In [12]:
city_names = []
for i in cities.findAll('a')[41:241]:
    city_names.append(i.getText())

In [13]:
city_names[:5]

['Tokyo', 'Delhi', 'Shanghai', 'Sao Paulo', 'Mexico City']

In [14]:
city_populations = []
for i in range(1000):
    if i%5 == 0:
        city_populations.append(cities.findAll('td')[i+2].getText())

In [15]:
city_populations[:5]

['37,393,129', '30,290,936', '27,058,479', '22,043,028', '21,782,378']

## Putting it all together

In [16]:
world_temperatures = []
for i in range(200):
    params['q'] = city_names[i]
    response = getCurrentWeather(params)
    if response.status_code != 200:
        continue
    city_information = response.json()
    country = city_information['sys']['country']
    latitude = city_information['coord']['lat']
    longitude = city_information['coord']['lon']
    weather_condition = city_information['weather'][0]['description']
    temperature = city_information['main']['feels_like']
    max_temp = city_information['main']['temp_max']
    min_temp = city_information['main']['temp_min']
    city_data = {'City': city_names[i],
                 'Country': country,
                 'Latitude': latitude,
                 'Longitude': longitude,
                 'Population': city_populations[i],
                 'Weather': weather_condition,
                 'Temperature (C)': temperature,
                 'Temperature Max (C)': max_temp,
                 'Temperature Min (C)': min_temp}
    world_temperatures.append(city_data)

world_temps_df = pd.DataFrame(world_temperatures, columns = city_data.keys())

In [17]:
world_temps_df

,City,Country,Latitude,Longitude,Population,Weather,Temperature (C),Temperature Max (C),Temperature Min (C)
0,Tokyo,JP,35.69,139.69,"37,393,129",light intensity shower rain,3.65,12.00,6.67
1,Delhi,IN,28.67,77.22,"30,290,936",haze,27.08,30.00,24.00
2,Shanghai,CN,31.22,121.46,"27,058,479",clear sky,7.69,12.00,11.00
3,Sao Paulo,BR,-23.55,-46.64,"22,043,028",broken clouds,15.71,22.22,16.00
4,Mexico City,MX,19.43,-99.13,"21,782,378",broken clouds,18.12,34.00,22.00
5,Dhaka,BD,23.71,90.41,"21,005,860",haze,30.63,27.00,27.00
6,Cairo,EG,30.06,31.25,"20,900,604",clear sky,14.44,16.00,15.56
7,Beijing,CN,39.91,116.40,"20,462,610",clear sky,10.41,17.22,10.00
8,Mumbai,IN,19.01,72.85,"20,411,274",haze,33.12,28.00,28.00
9,Osaka,JP,34.69,135.50,"19,165,340",light intensity shower rain,7.11,10.56,6.67


In [44]:
world_temps_df.to_csv('weather.csv')